In [1]:
import json

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
# Lấy file config
with open("config.json", "r") as jsonfile:
    config = json.load(jsonfile)
    print("Read file config successful")

Read file config successful


In [3]:
# demo lấy api rồi đẩy vào dataframe -> ghi vào hadoop
import requests
import json
from datetime import datetime

def get_stock_real_times_by_group (spark_instance, config):
    print(datetime.now())
    if config['method'] == 'POST':
        stock_real_times_by_group = requests.post(config['url'], json = config['body'])
    else: return
    result = json.loads(stock_real_times_by_group.text)['data']['stockRealtimesByGroup']
    df = spark_instance.sparkContext.parallelize(result).map(lambda x: json.dumps(x))
    df = spark_instance.read.json(df)
    # df.show()
    return df
    
#     df.write.format("json").mode("overwrite").save("hdfs://namenode:9000/project20221/raw/ssiDataApi.json")
#     checker = spark_instance.read.json("hdfs://namenode:9000/project20221/raw/ssiDataApi.json")
#     checker.printSchema()
#     checker.show(1, False)

In [4]:
from pyspark.sql.functions import to_timestamp, col, concat, lit

def transform_data (df, config):
    columns = []

    for column in config['Schema']:
        columns += [column['fieldName']]
        action = column['action']
        if action[0] == '/':
            df = df.withColumn(column['fieldName'], df[column['from']] / action[1:])
        elif action[0] == '*':
            df = df.withColumn(column['fieldName'], df[column['from']] * action[1:])
        elif action == 'timestamp':
            df = df.withColumn(column['fieldName'], to_timestamp(df[column['from']]))
        else:
            df = df.withColumn(column['fieldName'], df[column['from']])

        df = df.withColumn(column['fieldName'], df[column['fieldName']].cast(column['type']))

    return df.select(columns)

In [5]:
df = get_stock_real_times_by_group(spark, config)
print("Data before transform")
df.show(1)

2023-01-05 17:01:21.685498
Data before transform
+-------------+--------+---------+------------+-----------+--------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------------+---------------+--------+-------+-------------+---------------+--------+-----+-------+------+------------+-------------+----------------+-----------+------------------+--------+-----------------+---------------+----------------+-------+--------+-----------+---------+-------------+-----------+
|   __typename|avgPrice|best10Bid|best10BidVol|best10Offer|best10OfferVol|best1Bid|best1BidVol|best1Offer|best1OfferVol|best2Bid|best2BidVol|best2Offer|be

In [6]:
print("Data after transform")
df = transform_data(df, config)
df.show(5)
df.printSchema()

Data after transform
+--------+-------+------+
| stockNo|highest|lowest|
+--------+-------+------+
|hose:436|   23.0| 22.65|
|hose:217|   41.1| 40.25|
|hose:209|   48.4| 47.85|
|hose:580|  28.75| 28.25|
|hose:974|   81.5|  80.2|
+--------+-------+------+
only showing top 5 rows

root
 |-- stockNo: string (nullable = true)
 |-- highest: double (nullable = true)
 |-- lowest: double (nullable = true)

